In [ ]:
#%matplotlib

In [1]:
import shapefile
import numpy as np
import sys,os
import h5py
from matplotlib import pyplot as plt
from matplotlib import collections

sys.path.append(os.path.join(os.environ['ATS_SRC_DIR'],'tools','meshing_ats','meshing_ats'))

import meshing_ats
eps = 1.0

In [2]:
def dist(p0,p1):
    return np.linalg.norm(p0.x - p1.x)

def is_equal(p0,p1,eps=1.0):
    if (dist(p0,p1) < eps):
        return True
    return False

def centroid(points):
    return Point(sum(p.x for p in points)/len(points))

def ccw(A, B, C):
    """Tests whether the turn formed by A, B, and C is ccw"""
    return (B.x[0] - A.x[0]) * (C.x[1] - A.x[1]) > (B.x[1] - A.x[1]) * (C.x[0] - A.x[0])

def convex(points):
    """Returns 1 if convex, ccw ordered, -1 if convex but cw ordered, 0 if not convex"""
    is_ccw = []
    for i in range(len(points)):
        # Check every triplet of points
        A = points[i % len(points)]
        B = points[(i + 1) % len(points)]
        C = points[(i + 2) % len(points)]
        is_ccw.append(ccw(A,B,C))
    if all(is_ccw):
        return 1
    elif all([not(c) for c in is_ccw]):
        return -1
    return 0    

def star_convex(points):
    """Returns 1 if star-convex, ccw ordered, -1 if star-convex but cw ordered, 0 if not star-convex"""
    cent = centroid(points)
    convex_tris = []
    for i in range(len(points)):
        tri_points = [points[i], points[(i+1)%len(points)], cent]
        convex_tris.append(convex(tri_points))
    if all([cv_tri == 1 for cv_tri in convex_tris]):
        return 1
    elif all([cv_tri == -1 for cv_tri in convex_tris]):
        return -1
    return 0    

class Point(object):
    def __init__(self, x, decimals=1):
        self.x = np.array(x)
        self.xr = np.array([np.round(self.x[0],decimals), 
                            np.round(self.x[1],decimals)])
        
    def __hash__(self):
        return hash(tuple(self.xr))
        
    def __eq__(self, other):
        return is_equal(self, other, eps)
    
    def __repr__(self):
        return "Point: (%12.10g,%12.10g)"%(self.xr[0],self.xr[1])




In [3]:
# read the polygons via shapefile
sf_poly = shapefile.Reader("./Polygon_mesh_final/Polygon_mesh_final")
polygons = sf_poly.shapes()

nodes = []
conn = []

for s in polygons:
    poly_conn = []
    for p in s.points:
        myp = Point(p)
        if len(nodes) > 0:
            i = np.argmin(np.array([dist(myp, lcv) for lcv in nodes]))

            min_dist = dist(myp, nodes[i])
            if min_dist > eps:
                poly_conn.append(len(nodes))
                nodes.append(myp)
            else:
                poly_conn.append(i)
        else:
            nodes.append(myp)
            poly_conn.append(0)
    conn.append(poly_conn)
    
print (len(nodes))

xyz_nodes = np.zeros((len(nodes),3),'d')
for i,n in enumerate(nodes):
    xyz_nodes[i,0:2] = n.x

959


In [4]:
# write the nodes file
with open("./Polygon_mesh_nodes_final.csv",'w') as fid:
    fid.write("point ID,X,Y\n")
    for i,n in enumerate(nodes):
        fid.write("%d,%16.16g,%16.16g\n"%(i,n.x[0],n.x[1]))


In [5]:
# read the resulting shapefile with x,y,z nodes
sf_points = shapefile.Reader("./Polygon_mesh_nodes_sample_final/Polygon_mesh_nodes_sample_final")
shapes = sf_points.shapes()

xyz_points = np.array([[p.points[0][0], p.points[0][1], p.z[0]] for p in shapes])
points = [Point(xyz_points[i,0:2]) for i in range(xyz_points.shape[0])]

# match nodes to xyz_points
for i,n in enumerate(nodes):
    j_closest = np.argmin(np.array([dist(n,p) for p in points]))
    xyz_nodes[i,2] = xyz_points[j_closest,2]


In [6]:
# potentially strip duplicate points
for c in conn:
    if c[0] == c[-1]:
        c.pop()

In [7]:
%matplotlib 
# make the 2D mesh
import color
m2 = meshing_ats.Mesh2D(xyz_nodes,conn)
m2.plot(color=['b','r','g','m','c','y','gray'])
plt.show()

Using matplotlib backend: MacOSX


In [8]:
# check for non-star-convex cells
non_star_convex = []
for i,c in enumerate(conn):
    points = [nodes[j] for j in c]
    cv = star_convex(points)
    if cv == -1:
        c.reverse()
    elif cv == 1:
        # pass
        0 == 0
    else:
        print ("Non-star-convex cell:", i, c)
        plt.figure()
        c2 = c[:]
        c2.append(c2[0])
        coords = xyz_nodes[c2]
        plt.plot(coords[:,0],coords[:,1], 'b-x')
        cent = centroid(points)
        plt.scatter([cent.x[0],], [cent.x[1],], s=50, marker='s', c='r')
        non_star_convex.append(i)

fig,ax = plt.subplots(1,1)

m2.plot(color=['b','r','g','m','c'],ax=ax)

verts = [[xyz_nodes[i,0:2] for i in conn[p]] for p in non_star_convex]
gons = collections.PolyCollection(verts, facecolors='r')
ax.add_collection(gons)

plt.show()
print (gons)

In [9]:
#Use VisIt to get IDs of High-centered polygons, the GIDs should be obtained in parallel runs
path = '/Users/ajc/research/PreProcessing/python-scripts/meshes/watershed-barrow/'
def read_poly_ids():
    ids = []
    with open(os.path.join(path,'poly_char_gids/manually_char_polygons_new.txt')) as f:
        for line in f:
            if 'Zone:' in line:
                c = int(line.split()[1])
                if not c in ids:
                    ids.append(c)
    return ids


def get_ponded_depth():
    d1 = h5py.File('/Users/ajc/Projects/ATS-Data/OR-CONDO/ats-intermediate-ngee/surface_only/Rain_R1-serial/visdump_surface_data.h5','r')
    var = 'surface-ponded_depth.cell.0'
    keys = np.array(d1[var].keys(), 'i')
    keys = np.sort(keys, axis=None)
    cycle = '%s'%keys[-1]
    Ids_pd = d1[var][cycle][:]
    
    return Ids_pd
    

In [10]:
#IDS1= read_poly_ids()

In [11]:
IDS2= read_poly_ids()
#print (IDS2)

In [12]:
#id1=np.sort(IDS1)
hcp_gids=np.sort(IDS2)
#print (id1, hcp_gids)
print (len(hcp_gids),hcp_gids)

199 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  23  24  25  26  27  29  30  31  32  33  34  35  36  37  38
  45  46  47  48  49  50  52  53  54  55  56  58  59  60  61  62  63  64
  65  66  67  68  75  76  77  78  79  88  90  96  99 103 105 106 107 108
 110 114 117 118 119 120 121 128 129 131 152 154 155 163 166 167 177 178
 193 202 217 219 227 236 237 238 239 246 248 249 250 251 253 254 255 256
 257 258 259 260 261 262 264 265 266 270 274 275 276 277 278 279 280 281
 282 283 284 285 286 287 288 289 290 291 292 293 295 296 297 298 299 300
 314 315 316 317 318 319 322 324 325 327 328 329 330 331 332 333 334 335
 336 337 338 339 340 341 342 344 345 346 351 352 353 354 364 368 441 442
 443 444 445 446 447 448 449 450 451 452 453 457 458 460 461 464 465 466
 467]


In [13]:
#Changing organic layer thickness 
def org_layer_bottom_bndry(polygon, id):
    if (polygon == 'HCP' and id % 2 == 0):
        thickness = 0.1 
    elif(polygon == 'HCP'):
        thickness = 0.06 
    elif (polygon == 'LCP' and id %3 == 0):
        thickness = 0.2
    else:
        thickness = 0.16
    return thickness

def org_layer_bottom_bndry_gids(id):
    if id in hcp_gids:
        thickness = 0.1
    else:
        thickness= 0.2
        
    return thickness

def irz_layer_bottom_bndry_gids(id,d_thick):
    if id in hcp_gids:
        thickness = d_thick #0.4
    else:
        thickness= 0.0
    return thickness
def org_layer_bottom_bndry_pd(pd):
    pd = pd[0] * 100;
    #print pd
    if pd < 0.5:
        thickness = 8.
    elif (pd > 5.):
        thickness = 20
    elif (pd >=0.5 and pd<=5.0):
        m = (20. - 8.)/(5.- 0.5)
        y = 8. + m*(pd - 0.5)
        thickness = np.floor(y) - np.floor(y)%2
    return thickness/100.

In [17]:
#Ids = read_poly_ids()
#ponded_depth = get_ponded_depth()
gid_based = True

peat_thickness = np.zeros(m2.num_cells())
irz_thickness = np.zeros(m2.num_cells())
hcp_irz = 0.5
lcp_irz = 0.5



if gid_based == True:
    for i in range(m2.num_cells()):
        if i in hcp_gids:
            peat_thickness[i] = 0.1 #org_layer_bottom_bndry_gids(i)
            irz_thickness[i] = hcp_irz#irz_layer_bottom_bndry_gids(i,d_thick=0.0)
            #print ('HCP: ',i,peat_thickness[i])
        else:
            peat_thickness[i] = 0.2 #org_layer_bottom_bndry_gids(i)
            irz_thickness[i] = lcp_irz #irz_layer_bottom_bndry_gids(i,d_thick=0.56)
            #print ('LCP: ',i,peat_thickness[i])
else:
    for i in range(m2.num_cells()):
        peat_thickness[i] = org_layer_bottom_bndry_pd(ponded_depth[i])
        #print (i, peat_thickness[i], ponded_depth[i]*100)

XX = [0.02,0.04,0.08,0.12,0.16,0.2]
import itertools

def get_next(iterable):
    for item in itertools.cycle(iterable):
        yield item

pt = get_next(XX)
for i in range(m2.num_cells()):
    peat_thickness[i] = next(pt)
    irz_thickness[i] = 0.5
    
print (peat_thickness)
#print (irz_thickness)

[0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04
 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12
 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2
 0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04
 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12
 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2
 0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04
 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12
 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2
 0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04
 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12
 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2
 0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04
 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12 0.16 0.2  0.02 0.04 0.08 0.12
 0.16 0.2 

In [ ]:

# Ice rich zone 50 cm
#variable peat thickness
def make_mesh():
    outfile = path
    # layer extrusion
    layer_types = []
    layer_data = []
    layer_ncells = []
    layer_mat_ids = []


    z=0
    Z = []

    for i in range(8):#10 
        layer_types.append('constant')
        layer_data.append(0.02)
        layer_ncells.append(1)
        layer_mat_ids.append(-10000*np.ones((m2.num_cells(),),'i'))
        z = round(z + 0.02, 6)
        Z.append(z)
    print ('Peat ', z)

    for i in range(17): #8cm peat, n=20, 20cm peat n = 14
        layer_types.append('constant')
        layer_data.append(0.02)
        layer_ncells.append(1)
        layer_mat_ids.append(1003*np.ones((m2.num_cells(),),'i'))
        z = round(z + 0.02, 6)
        Z.append(z)
    print ('Upper mineral ', z)


    dz = .02
    for i in range(30):
        dz *= 1.075
        layer_types.append("constant")
        layer_data.append(dz)
        layer_ncells.append(1)
        layer_mat_ids.append(1004*np.ones((m2.num_cells(),),'i'))
        z = round(z + dz,6)
        Z.append(z)
    print ('Ice rich', z)

    for i in range(25):
        dz *= 1.14
        layer_types.append("constant")
        layer_data.append(dz)
        layer_ncells.append(1)
        layer_mat_ids.append(1005*np.ones((m2.num_cells(),),'i'))
        z = round(z + dz,6)
        Z.append(z)
    print (z)



    layer_types.append('snapped')
    layer_data.append(-45.0) # bottom location
    layer_ncells.append(1)
    layer_mat_ids.append(1005*np.ones((m2.num_cells(),),'i'))


    mat_ids=np.zeros((m2.num_cells(), 9), 'i')
    #mat_ids=np.zeros((m2.num_cells(), 11), 'i')

    for i in range(m2.num_cells()):
        for j in range(9):
            if j == 0 :
                mat_ids[i,j]=1001
            elif (Z[j] <= peat_thickness[i]):
                #print (i,j,Z[j],peat_thickness[i])
                mat_ids[i,j]=1002
            else:
                mat_ids[i,j]=1003
        #break
    for j in range(9):
        layer_mat_ids[j] = mat_ids[:,j]

    print ('HERE ',len(layer_mat_ids), len(layer_ncells), len(layer_types), len(layer_data))
    #print (Z)
    #print (layer_mat_ids[8])

    m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                            layer_data, 
                                            layer_ncells, 
                                            layer_mat_ids)
    print ('testing.exo'.split("."))
    #m3.write_exodus("testing.exo")#"barrow_polygon_watershed_5layer-icerich50cm-omvar.exo")
    #m3.write_exodus(os.path.join(outfile + 'barrow_polygon_watershed_5layer-icerich50cm-omvar.exo').encode('utf8'))
    m3.write_exodus('barrow_polygon_watershed_5layer-icerich50cm-omvar.exo'.encode('utf8'))

In [24]:
# Ice rich zone 40 cm for LCP, and 50 cm for HCP
#variable peat thickness
# layer extrusion
def make_mesh_variable_irz_om():
    layer_types = []
    layer_data = []
    layer_ncells = []
    layer_mat_ids = []

    z=0
    Z = []
    peat_layer_cells = 10
    var_layer_cells_irz = 20 #26 cells covers 80 cm
    #irz_start_cell_id = var_layer_cells + var_layer_cells_irz
    irz_start_cell=1 #10 for irz=40, 16
    
    Ncells = 0
    for i in range(peat_layer_cells):
        layer_types.append('constant')
        layer_data.append(0.02)
        layer_ncells.append(1)
        layer_mat_ids.append(-10000*np.ones((m2.num_cells(),),'i'))
        z = round(z + 0.02, 6)
        Z.append(z)
        Ncells +=1
    print ('Organic matter thickness: ', Ncells, z)

    for i in range(10): 
        layer_types.append('constant')
        layer_data.append(0.02)
        layer_ncells.append(1)
        layer_mat_ids.append(1003*np.ones((m2.num_cells(),),'i'))
        z = round(z + 0.02, 6)
        Z.append(z)
        Ncells +=1
    print ('Upper mineral 1: ', Ncells, z)

    dz = .02 #keep the top 60 cm to 2cm resolution
    for i in range(var_layer_cells_irz):
        #dz *= 1.025
        dz = 0.02
        dz = np.round(dz,4)
        layer_types.append("constant")
        layer_data.append(dz)
        layer_ncells.append(1)
        layer_mat_ids.append(-1004*np.ones((m2.num_cells(),),'i'))
        z = round(z + dz,6)
        Z.append(z)
        Ncells +=1
    print ('Ice rich zone 1: ', Ncells, z, Z[-1])

    #dz = .02
    for i in range(20):
        dz *= 1.125
        dz = np.round(dz,4)
        layer_types.append("constant")
        layer_data.append(dz)
        layer_ncells.append(1)
        layer_mat_ids.append(1004*np.ones((m2.num_cells(),),'i'))
        z = round(z + dz,6)
        Z.append(z)
        Ncells +=1
    print ('Ice rich zone 2: ', Ncells, z, Z[-1])

    for i in range(19):
        dz *= 1.2
        dz = np.round(dz,4)
        layer_types.append("constant")
        layer_data.append(dz)
        layer_ncells.append(1)
        layer_mat_ids.append(1005*np.ones((m2.num_cells(),),'i'))
        z = round(z + dz,6)
        Z.append(z)
    print ('Mineral ',z, Z[-1])


    layer_types.append('snapped')
    layer_data.append(-45.0) # bottom location
    layer_ncells.append(1)
    layer_mat_ids.append(1005*np.ones((m2.num_cells(),),'i'))

    #---------------------------------------------------------------
    # Assign variable peat layers
    mat_ids=np.zeros((m2.num_cells(), peat_layer_cells), 'i')
    test_id =-1
    for i in range(m2.num_cells()):
        for j in range(peat_layer_cells):
            if j == 0 :
                mat_ids[i,j]=1001
            elif (Z[j] <= peat_thickness[i]):
                #print (i,j,Z[j],peat_thickness[i])
                mat_ids[i,j]=1002
                test_id=1002
            else:
                mat_ids[i,j]=1003
                test_id=1003
        #print (i,peat_thickness[i],test_id)
    #print ('HCP: ', hcp_gids[44], layer_mat_ids[9][214])
    for j in range(peat_layer_cells):
        
        layer_mat_ids[j] = mat_ids[:,j]
        #print ('J: ',j, layer_mat_ids[j])
    
    print ('HCP: ', hcp_gids[44], layer_mat_ids[8][58])
    
    #---------------------------------------------------------------
    #for i in range(37):
    #    print (i, layer_mat_ids[i][261])
    #---------------------------------------------------------------
    # Assign variable ice rich zone layers
    mat_ids=np.zeros((m2.num_cells(), var_layer_cells_irz), 'i')
    for i in range(m2.num_cells()):
        #print ('TOP', i, hcp_gids[i])
        for j in range(var_layer_cells_irz):
            if (Z[20+j] > irz_thickness[i]):
                #print ('IRZ',i,j,Z[19+j],irz_thickness[i])
                mat_ids[i,j]=1004
            else:
                #print ('No IRZ ',i,j,Z[20+j],irz_thickness[i])
                mat_ids[i,j]=1003

    for j in range(20,20+var_layer_cells_irz):#starts at 20 due to 40 cm depth of ice rich zone
        layer_mat_ids[j] = mat_ids[:,j-20]
    
    #---------------------------------------------------------------
    #for i in range(37):
    #    print ('A: ',Z[i], i, layer_mat_ids[i][262])
    #print ('HERE ',len(layer_mat_ids), len(layer_ncells), len(layer_types), len(layer_data))
    #print (Z)
    #print (layer_mat_ids)

    m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                            layer_data, 
                                            layer_ncells, 
                                            layer_mat_ids)#,hcp_gids)

    #m3.write_exodus('barrow_polygon_watershed_5layer-var_irz5050_var_om_test.exo'.encode('utf8'))
    m3.write_exodus('barrow_polygon_watershed_5layer-var_irz5050_var_om_movie.exo'.encode('utf8'))

In [25]:
make_mesh_variable_irz_om()

Organic matter thickness:  10 0.2
Upper mineral 1:  20 0.4
Ice rich zone 1:  40 0.8 0.8
Ice rich zone 2:  60 2.519 2.519
Mineral  41.6886 41.6886
HCP:  54 1003

You are using exodus.py v 1.16 (seacas-py3), a python wrapper of some of the exodus library.

Copyright (c) 2013, 2014, 2015, 2016, 2017, 2018, 2019 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.

BASENAME_FILE_Exodus:  barrow_polygon_watershed_5layer-var_irz5050_var_om_movie.exo
barrow_polygon_watershed_5layer-var_irz5050_var_om_movie
Opening exodus file: barrow_polygon_watershed_5layer-var_irz5050_var_om_movie.exo
Closing exodus file: barrow_polygon_watershed_5layer-var_irz5050_var_om_movie.exo


In [ ]:
#write subgrid parameters to .h5 file
import h5py
infile = h5py.File('/Users/ajc/FUSE/simulations/production/old/polygonal_tundra.new/data_sg_para/sg_para-watershed468-new.h5','r')
outfile = h5py.File('/Users/ajc/FUSE/simulations/production/barrow_watershed/data/sg_parameters/sg_para-watershed468_1204.h5','w')
#hcp_gids
dmax = np.ones(468)*0.361
dex = np.ones(468)*0.361/2
delta = np.ones(468)*0.15
beta = np.ones(468)*7.0
for h in hcp_gids:
    dmax[h] = 0.416
    dex[h] = 0.228
    delta[h] = 0.08
    beta[h] = 1.5
for key in infile.keys():
    if not 'time' in key:
        d = infile[key]['0'][:1]
        grp = outfile.create_group(key)
        if 'microtopographic_relief' in key:
            grp.create_dataset('0',data=dmax)
        if 'excluded_volume' in key:
            grp.create_dataset('0',data=dex)
        if 'depression_depth' in key:
            grp.create_dataset('0',data=delta)
        if 'drag_exponent' in key:
            grp.create_dataset('0',data=beta)
    else:
        outfile.create_dataset('time',data=infile[key])
        #print (key,d[:4])
outfile.close()

In [ ]:
from scipy.spatial import ConvexHull
points = np.random.rand(4, 2)   # 30 random points in 2-D
hull = ConvexHull(points)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(points[:,0], points[:,1], 'o')
for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')